# PNL - Verificando as expressões mais importantes na avaliação de hotéis

### Utilizando o repositório do UCI 
https://archive.ics.uci.edu/ml/datasets/Eco-hotel

In [51]:
import pandas as pd
import nltk
import spacy
import re
import string
from nltk.corpus import stopwords

In [14]:
dataset = pd.read_csv("dataset.csv", sep='delimiter')

C:\Users\mateu\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [110]:
dataset.isna().sum()

Review    0
dtype: int64

In [49]:
comentario = dataset['Review']

In [50]:
def removeNoAscii(s): 
    return "".join(i for i in s if ord(i) < 128)

comentario = comentario.map(lambda x: removeNoAscii(x))

In [53]:
dicionario_stopwords = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}

In [62]:
def descobre_idioma(text):
    
    # Aplica tokenização considerando pontuação
    palavras = set(nltk.wordpunct_tokenize(text.lower()))
    
    # Conta o total de palavras tokenizadas considerando o dicionário de stopwords
    lang = max(((lang, len(palavras & stopwords)) for lang, stopwords in dicionario_stopwords.items()), key = lambda x: x[1])[0]
    
    # Verifica se o idioma é português
    
    if lang == 'english':
        return True
    else:
        return False

comentarios = comentario[comentario.apply(descobre_idioma)]
#Todas as classificações estão em inglês

In [63]:
# Carrega o dcionário em nossa sessão SpaCy
nlp = spacy.load("en_core_web_sm")

In [64]:
# Função para limpar e lematizar os comentários
def limpa_comentarios(text):
    
    # Remove pontuação usando expressão regular
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    
    # Usa o SpaCy para lematização
    doc = nlp(nopunct, disable = ['parser', 'ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [73]:
# Aplica a função aos dados
comentarios_ingles_lemmatized = comentarios.map(limpa_comentarios)

# Coloca tudo em minúsculo
comentarios_ingles_lemmatized = comentarios_ingles_lemmatized.map(lambda x: [word.lower() for word in x])

#Tokenizando
comentarios_tokens = [item for items in comentarios_ingles_lemmatized for item in items]

token = pd.DataFrame(comentarios_tokens)
token

,0
0,everything
1,from
2,the
3,weather
4,
...,...
21661,with
21662,-pron-
21663,family
21664,again


## Separando em Bigramas

In [101]:
buscaBigramas = nltk.collocations.BigramCollocationFinder.from_words(comentarios_tokens)
bigrama_freq = buscaBigramas.ngram_fd.items()
FreqTabBigramas = pd.DataFrame(list(bigrama_freq), columns = ['Bigrama', 'Freq']).sort_values(by = 'Freq', ascending = False)
FreqTabBigramas

,Bigrama,Freq
101,"( , -pron-)",377
24,"( , the)",244
27,"(-pron-, be)",195
358,"(thank, -pron-)",135
110,"(-pron-, have)",116
...,...,...
5067,"(team, jose)",1
5066,"(all, area)",1
5065,"(huge, to)",1
5064,"(and, huge)",1


In [113]:
# Vamos criar uma lista de stopwords
en_stopwords = set(stopwords.words('english'))

def filtra_tipo_token_bigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [114]:
bigramas_filtrados = FreqTabBigramas[FreqTabBigramas.Bigrama.map(lambda x: filtra_tipo_token_bigrama(x))]
bigramas_filtrados.head()

,Bigrama,Freq
888,"(magical, place)",15
623,"(special, place)",14
360,"(wonderful, stay)",13
301,"(amazing, place)",11
1473,"(wonderful, place)",11


## Separando em Trigramas

In [120]:
buscaTrigramas = nltk.collocations.TrigramCollocationFinder.from_words(comentarios_tokens)
trigrama_freq = buscaTrigramas.ngram_fd.items()
FreqTabTrigramas = pd.DataFrame(list(trigrama_freq), columns = ['Trigrama', 'Freq']).sort_values(by = 'Freq', ascending = False)
FreqTabTrigramas

,Trigrama,Freq
103,"( , -pron-, be)",75
504,"( , thank, -pron-)",62
401,"(thank, -pron-, for)",54
268,"( , -pron-, will)",36
336,"( , -pron-, have)",35
...,...,...
6564,"(to, nyc, -pron-)",1
6565,"(nyc, -pron-, speak)",1
6566,"(-pron-, speak, with)",1
6567,"(speak, with, an)",1


In [123]:
en_stopwords = set(stopwords.words('english'))

def filtra_tipo_token_trigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in first_type and tags[2][1] in second_type:
        return True
    else:
        return False

In [124]:
trigramas_filtrados = FreqTabTrigramas[FreqTabTrigramas.Trigrama.map(lambda x: filtra_tipo_token_trigrama(x))]

,Trigrama,Freq
1119,"(chef, leonardo, pereira)",7
3778,"(dream, come, true)",4
2625,"(everything, 5, star)",2
16326,"(many, thank, everybody)",2
12277,"(vegetarian, main, course)",2
...,...,...
6715,"(small, birth, rate)",1
6685,"(spend, 40, year)",1
6470,"(memorable, dining, experience)",1
6595,"(beautiful, every, time)",1


## Bigramas e Trigramas mais usados

In [167]:
fraseBigrama = bigramas_filtrados['Bigrama'].head(5).values
fraseBigrama = pd.DataFrame([str(i[0]+' '+i[1]) for i in fraseBigrama],columns=['Bigramas'])
fraseBigrama

,Bigramas
0,magical place
1,special place
2,wonderful stay
3,amazing place
4,wonderful place


In [171]:
fraseTrigrama = trigramas_filtrados['Trigrama'].head(5).values
fraseTrigrama = pd.DataFrame([str(i[0]+' '+i[1]+' '+i[2]) for i in fraseTrigrama],columns=['Trigramas'])
fraseTrigrama

,Trigramas
0,chef leonardo pereira
1,dream come true
2,everything 5 star
3,many thank everybody
4,vegetarian main course


In [207]:
Tabela = pd.concat([fraseBigrama,bigramas_filtrados['Freq'].head(5).reset_index(drop=True),
                    fraseTrigrama,trigramas_filtrados['Freq'].head(5).reset_index(drop=True)],axis=1)
Tabela

,Bigramas,Freq,Trigramas,Freq
0,magical place,15,chef leonardo pereira,7
1,special place,14,dream come true,4
2,wonderful stay,13,everything 5 star,2
3,amazing place,11,many thank everybody,2
4,wonderful place,11,vegetarian main course,2
